In [ ]:
#import snappy

In [1]:
from snappy import HashMap
from snappy import ProductIO
from snappy import jpy


In [2]:
import os,gc
from snappy import GPF

In [3]:
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
Hashmap = jpy.get_type('java.util.HashMap')

In [40]:
path = "data/input/"
opath = "data/output/"
folder = "S1A_IW_GRDH_1SDV_20190821T004329_20190821T004354_028660_033E51_6F6F.SAFE"
gc.enable()
imagepath = path + folder + "\\"
output = opath + "\\"
timestamp = folder.split('_')[4]
date = timestamp[:8]

s1_org = ProductIO.readProduct(imagepath + "\\manifest.safe")
print(s1_org)



orbit = s1_org.getMetadataRoot().getElement('Abstracted_Metadata').getAttribute('PASS').getData()
look = s1_org.getMetadataRoot().getElement('Abstracted_Metadata').getAttribute('antenna_pointing').getData()

print(orbit)
print (look)


org.esa.snap.core.datamodel.Product[name=S1A_IW_GRDH_1SDV_20190821T004329_20190821T004354_028660_033E51_6F6F]
DESCENDING
right


In [41]:
width = s1_org.getSceneRasterWidth()
height = s1_org.getSceneRasterHeight()
bands = s1_org.getBandNames()
print (width)
print (height)
print (list(bands))

25381
16756
['Amplitude_VH', 'Intensity_VH', 'Amplitude_VV', 'Intensity_VV']


In [42]:
# first calibrate then subset -

pols = ['VH'] #,'VV']
    
for p in pols:
    polarization = p
        
    parameters = HashMap()
    parameters.put('outputSigmaBand',True)
    parameters.put('sourceBands','Intensity_'+ polarization)
    parameters.put('selectedPolarizations' , polarization)
    parameters.put('outputImageScaleInDb', False)
        
    s1_calib = GPF.createProduct('Calibration',parameters,s1_org)
        
    #s1_calib_bd = output + date + "_calibrate_" + polarization
        
    #ProductIO.writeProduct(s1_calib,s1_calib_bd, 'BEAM-DIMAP')
        
    print (s1_calib)
    
    WKTReader = jpy.get_type('com.vividsolutions.jts.io.WKTReader')

    lon_min = 77.5
    lat_min = 30.5
    lon_max = 78.5
    lat_max = 31.5

#wkt = "POLYGON((lon_min lat_min,lon_max lat_min ,lon_max lat_max ,lon_min lat_max ,lon_min lat_min))"
#wkt = "POLYGON((77.5 30.5, 78.5 30.5, 78.5 31.5 , 77.5 31.5 , 77.5 30.5))"
    wkt = "POLYGON((77.5 30.5, 77.6 30.5, 77.6 30.6 , 77.5 30.6 , 77.5 30.5))"
    geom = WKTReader().read(wkt)

    parameters = HashMap()
    parameters.put('copyMeta', True)
    parameters.put('geoRegion', geom)
    parameters.put('outputImageScaleinDB', False)

    s1_subset = GPF.createProduct('Subset',parameters,s1_calib)

    #ProductIO.
    print (s1_subset)


    orbit = s1_subset.getMetadataRoot().getElement('Abstracted_Metadata').getAttribute('PASS').getData()
    look = s1_subset.getMetadataRoot().getElement('Abstracted_Metadata').getAttribute('antenna_pointing').getData()

       ### TERRAIN CORRECTION
 
    parameters = HashMap()     
    parameters.put('demResamplingMethod', 'NEAREST_NEIGHBOUR') 
    parameters.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR') 
    parameters.put('demName', 'SRTM 3Sec') 
    parameters.put('pixelSpacingInMeter', 10.0) 
    parameters.put('sourceBands', 'Sigma0_' + polarization)
 
    terrain = output + date + "_corrected_" + polarization 
    target_2 = GPF.createProduct("Terrain-Correction", parameters, s1_subset) 
    ProductIO.writeProduct(target_2, terrain, 'GeoTIFF')
    
    
    

print(orbit)
print (look)



del s1_org
del s1_subset


org.esa.snap.core.datamodel.Product[name=S1A_IW_GRDH_1SDV_20190821T004329_20190821T004354_028660_033E51_6F6F_Cal]
org.esa.snap.core.datamodel.Product[name=Subset_S1A_IW_GRDH_1SDV_20190821T004329_20190821T004354_028660_033E51_6F6F_Cal]
DESCENDING
right


In [46]:
import rasterio as rio


ImportError: No module named 'rasterio'

In [45]:
# first subset then calibrate - for some reason the calibration element is not getting copied in the subset


In [43]:
print(list(s1_subset.getBandNames()))
print(s1_subset.getSceneRasterWidth())

NameError: name 's1_subset' is not defined

In [44]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

%matplotlib inline

def plotBand(product, band, vmin, vmax):
     
    band = product.getBand(band)

    w = band.getRasterWidth()
    h = band.getRasterHeight()

    band_data = np.zeros(w * h, np.float32)
    band.readPixels(0, 0, w, h, band_data)

    band_data.shape = h, w

    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    imgplot = plt.imshow(band_data, cmap=plt.cm.binary, vmin=vmin, vmax=vmax)
    
    return imgplot 

s1_subset = s1_org = ProductIO.readProduct(output + "20190821_corrected_VH.tif")
plotBand(s1_subset, 'Amplitude_VV', 0, 750)



AttributeError: 'NoneType' object has no attribute 'getRasterWidth'